# Using CAVE with AutoPyTorch

AutoPyTorch aims at building a framework for automated neural-network-configuration. Currently it supports [BOHB](https://github.com/automl/HpBandSter) for hyperparameter search.
CAVE integrates AutoPyTorch, building on it's function for further insights and visualizations.
This notebook provides an exemplary pipeline for using CAVE on / with AutoPyTorch.

We will generate some AutoPyTorch-Output. You can use your own AutoPyTorch-routine here, we will use the openml-tasks, inspired by [AutoPyTorch's tutorial notebook](https://github.com/automl/Auto-PyTorch/blob/master/examples/basics/Auto-PyTorch%20Tutorial.ipynb).

In [1]:
import shutil
log_dir = "logs/apt-cave-notebook/"
shutil.rmtree(log_dir, ignore_errors=True)
import logging
logging.basicConfig(level=logging.DEBUG)

In [2]:
from autoPyTorch import AutoNetClassification
import pandas as pd
import numpy as np
import os as os
import openml
import json
from ConfigSpace.read_and_write import json as pcs_json
# Logging
from autoPyTorch.components.metrics.additional_logs import *
from autoPyTorch.pipeline.nodes import LogFunctionsSelector

task = openml.tasks.get_task(task_id=31)

X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

autopytorch = AutoNetClassification(config_preset="medium_cs",
                                    result_logger_dir=log_dir,
                                    log_every_n_datapoints=10,
                                    use_tensorboard_logger=True,
                                    additional_logs=[test_result.__name__,
                                                     test_cross_entropy.__name__,
                                                     test_balanced_accuracy.__name__],
                                   )

# Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
task = openml.tasks.get_task(task_id=31)
X, y = task.get_X_and_y()
ind_train, ind_test = task.get_train_test_split_indices()
X_train, Y_train = X[ind_train], y[ind_train]
X_test, Y_test = X[ind_test], y[ind_test]

DEBUG:matplotlib:$HOME=/home/shuki
DEBUG:matplotlib:CONFIGDIR=/home/shuki/.config/matplotlib
DEBUG:matplotlib:matplotlib data path: /home/shuki/VirtualEnvs/CAVE_dev/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/shuki/VirtualEnvs/CAVE_dev/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.1.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['builtins', 'sys', '_frozen_importlib', '_imp', '_warnings', '_thread', '_weakref', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'zipimport', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_weakrefset', '_bootlocale', '_locale', 'site', 'os', 'errno', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', 'types', 'functools', '_functools', 'collections', 'operator', '_operator', 'keyword', 

DEBUG:matplotlib:CACHEDIR=/home/shuki/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/shuki/.cache/matplotlib/fontlist-v310.json
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


In [3]:
# Equip autopytorch with additional logs
gl = GradientLogger()
lw_gl = LayerWiseGradientLogger()
additional_logs = [gradient_max(gl), gradient_mean(gl), gradient_median(gl), gradient_std(gl),
                   gradient_q10(gl), gradient_q25(gl), gradient_q75(gl), gradient_q90(gl),
                   layer_wise_gradient_max(lw_gl), layer_wise_gradient_mean(lw_gl),
                   layer_wise_gradient_median(lw_gl), layer_wise_gradient_std(lw_gl),
                   layer_wise_gradient_q10(lw_gl), layer_wise_gradient_q25(lw_gl),
                   layer_wise_gradient_q75(lw_gl), layer_wise_gradient_q90(lw_gl),
                   gradient_norm()]

for additional_log in additional_logs:
    autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=type(additional_log).__name__,
                                                                       log_function=additional_log)

    #sampling_space["additional_logs"].append(type(additional_log).__name__)

autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_result.__name__, 
                                                                   log_function=test_result(autopytorch, X[ind_test], y[ind_test]))
autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_cross_entropy.__name__,
                                                                   log_function=test_cross_entropy(autopytorch, X[ind_test], y[ind_test]))
autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_balanced_accuracy.__name__,
                                                                   log_function=test_balanced_accuracy(autopytorch, X[ind_test], y[ind_test]))


In [4]:
# Fit to find an incumbent configuration with BOHB
results_fit = autopytorch.fit(X_train=X_train,
                              Y_train=Y_train,
                              validation_split=0.3,
                              max_runtime=75,
                              min_budget=10,
                              max_budget=50,
                              refit=True,
                             )
autopytorch.refit_all_incumbents(X_train, Y_train)

DEBUG:autonet:Fit: AutoNetSettings


TRAINER: Logging at step 173 , counter is at 173
TRAINER: Points seen in current epoch 173 of 900


Process pynisher function call:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shuki/VirtualEnvs/CAVE_dev/lib/python3.6/site-packages/pynisher/limit_function_call.py", line 93, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/core/worker.py", line 124, in optimize_pipeline
    raise e
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/core/worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/pipeline/base/pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/pipeline/base/node.py", line 115, in fit

TRAINER: Logging at step 245 , counter is at 245
TRAINER: Points seen in current epoch 245 of 900


Process pynisher function call:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/shuki/VirtualEnvs/CAVE_dev/lib/python3.6/site-packages/pynisher/limit_function_call.py", line 93, in subprocess_func
    return_value = ((func(*args, **kwargs), 0))
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/core/worker.py", line 124, in optimize_pipeline
    raise e
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/core/worker.py", line 118, in optimize_pipeline
    refit=False, rescore=False, hyperparameter_config_id=config_id, dataset_info=self.dataset_info)
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/pipeline/base/pipeline.py", line 60, in fit_pipeline
    return self.root.fit_traverse(**kwargs)
  File "/home/shuki/Repos/APT_shukon/autoPyTorch/pipeline/base/node.py", line 115, in fit

...

TRAINER: Logging at step 4194 , counter is at 3874
TRAINER: Points seen in current epoch 414 of 1260
TRAINER: Loss/Acc.: 0.7187172571818033 56.666666666666664
TRAINER: Logging at step 4332 , counter is at 4002
TRAINER: Points seen in current epoch 552 of 1260
TRAINER: Loss/Acc.: 0.7156051993370056 56.904761904761905
TRAINER: Logging at step 4470 , counter is at 4130
TRAINER: Points seen in current epoch 690 of 1260
TRAINER: Loss/Acc.: 0.7149809956550598 56.58730158730159
TRAINER: Logging at step 4608 , counter is at 4258
TRAINER: Points seen in current epoch 828 of 1260
TRAINER: Loss/Acc.: 0.7093351781368256 56.666666666666664
TRAINER: Logging at step 4746 , counter is at 4386
TRAINER: Points seen in current epoch 966 of 1260
TRAINER: Loss/Acc.: 0.7095787099429539 55.87301587301587
TRAINER: Logging at step 4884 , counter is at 4514
TRAINER: Points seen in current epoch 1104 of 1260
TRAINER: Loss/Acc.: 0.7078083828091621 51.507936507936506
TRAINER: Logging at step 5022 , counter is at 4

TRAINER: Logging at step 10770 , counter is at 9930
TRAINER: Points seen in current epoch 690 of 1260
TRAINER: Loss/Acc.: 0.6920834898948669 55.47619047619048
TRAINER: Logging at step 10908 , counter is at 10058
TRAINER: Points seen in current epoch 828 of 1260
TRAINER: Loss/Acc.: 0.6913191378116608 57.22222222222222
TRAINER: Logging at step 11046 , counter is at 10186
TRAINER: Points seen in current epoch 966 of 1260
TRAINER: Loss/Acc.: 0.6914211937359401 56.904761904761905
TRAINER: Logging at step 11184 , counter is at 10314
TRAINER: Points seen in current epoch 1104 of 1260
TRAINER: Loss/Acc.: 0.6887978464365005 56.904761904761905
TRAINER: Logging at step 11322 , counter is at 10442
TRAINER: Points seen in current epoch 1242 of 1260
TRAINER: Loss/Acc.: 0.6881877581278483 57.06349206349206
TRAINER: Logging at step 11340 , counter is at 10450
TRAINER: Points seen in current epoch 1260 of 1260
TRAINER: Loss/Acc.: 0.6893391030175345 55.55555555555556
TRAINER: Logging at step 11478 , cou

TRAINER: Logging at step 17346 , counter is at 15986
TRAINER: Points seen in current epoch 966 of 1260
TRAINER: Loss/Acc.: 0.6790379200662885 55.31746031746032
TRAINER: Logging at step 17484 , counter is at 16114
TRAINER: Points seen in current epoch 1104 of 1260
TRAINER: Loss/Acc.: 0.6801433637738228 55.31746031746032
TRAINER: Logging at step 17622 , counter is at 16242
TRAINER: Points seen in current epoch 1242 of 1260
TRAINER: Loss/Acc.: 0.6789566609594557 54.920634920634924
TRAINER: Logging at step 17640 , counter is at 16250
TRAINER: Points seen in current epoch 1260 of 1260
TRAINER: Loss/Acc.: 0.6788482702913738 55.3968253968254
TRAINER: Logging at step 17778 , counter is at 16378
TRAINER: Points seen in current epoch 138 of 1260
TRAINER: Loss/Acc.: 0.6772025227546692 55.31746031746032
TRAINER: Logging at step 17916 , counter is at 16506
TRAINER: Points seen in current epoch 276 of 1260
TRAINER: Loss/Acc.: 0.6869822144508362 55.3968253968254
TRAINER: Logging at step 18054 , count

TRAINER: Logging at step 23922 , counter is at 22042
TRAINER: Points seen in current epoch 1242 of 1260
TRAINER: Loss/Acc.: 0.6830615467495389 54.84126984126985
TRAINER: Logging at step 23940 , counter is at 22050
TRAINER: Points seen in current epoch 1260 of 1260
TRAINER: Loss/Acc.: 0.6835819752443404 54.84126984126985
TRAINER: Logging at step 24078 , counter is at 22178
TRAINER: Points seen in current epoch 138 of 1260
TRAINER: Loss/Acc.: 0.6761606335639954 54.84126984126985
TRAINER: Logging at step 24216 , counter is at 22306
TRAINER: Points seen in current epoch 276 of 1260
TRAINER: Loss/Acc.: 0.6742779314517975 57.38095238095238
TRAINER: Logging at step 24354 , counter is at 22434
TRAINER: Points seen in current epoch 414 of 1260
TRAINER: Loss/Acc.: 0.6746296683947245 56.507936507936506
TRAINER: Logging at step 24492 , counter is at 22562
TRAINER: Points seen in current epoch 552 of 1260
TRAINER: Loss/Acc.: 0.6760445237159729 56.666666666666664
TRAINER: Logging at step 24630 , cou

TRAINER: Logging at step 30378 , counter is at 27978
TRAINER: Points seen in current epoch 138 of 1260
TRAINER: Loss/Acc.: 0.7089474201202393 50.0
TRAINER: Logging at step 30516 , counter is at 28106
TRAINER: Points seen in current epoch 276 of 1260
TRAINER: Loss/Acc.: 0.7115265130996704 50.0
TRAINER: Logging at step 30654 , counter is at 28234
TRAINER: Points seen in current epoch 414 of 1260
TRAINER: Loss/Acc.: 0.6955951452255249 50.0
TRAINER: Logging at step 30792 , counter is at 28362
TRAINER: Points seen in current epoch 552 of 1260
TRAINER: Loss/Acc.: 0.693925142288208 50.0
TRAINER: Logging at step 30930 , counter is at 28490
TRAINER: Points seen in current epoch 690 of 1260
TRAINER: Loss/Acc.: 0.6930542349815368 50.317460317460316
TRAINER: Logging at step 31068 , counter is at 28618
TRAINER: Points seen in current epoch 828 of 1260
TRAINER: Loss/Acc.: 0.6903521120548248 50.476190476190474
TRAINER: Logging at step 31206 , counter is at 28746
TRAINER: Points seen in current epoch 

In [5]:
# Save fit results as json
with open(os.path.join(log_dir, "results_fit.json"), "w") as f:
    json.dump(results_fit, f, indent=2)
    
# Also necessary information (can be migrated either to CAVE or (preferably) to autopytorch)
with open(os.path.join(log_dir, 'configspace.json'), 'w') as f:
    f.write(pcs_json.write(autopytorch.get_hyperparameter_search_space(X_train=X_train,
                                                                   Y_train=Y_train)))
with open(os.path.join(log_dir, 'autonet_config.json'), 'w') as f:
    json.dump(autopytorch.get_current_autonet_config(), f, indent=2)

We can then spin up CAVE and hand it the output, as well as the autonet-instance. That way, CAVE can refit the incumbents and we can investigate the evolution of the network a bit closer.

In [6]:
from cave.cavefacade import CAVE

cave_output_dir = "cave_output"

autopytorch.update_autonet_config(autonet_config=dict([('result_logger_dir', cave_output_dir)]))

# The information in the autonet-bundle needs to be logged and loaded eventually (or all necessary logging reliably triggered in apt itself)
autonet_bundle = {'autopytorch': autopytorch,
                  'X_train': X_train,
                  'Y_train': Y_train,
                 }

cave = CAVE([log_dir],        # List of folders holding results
            cave_output_dir,  # Output directory
            ['.'],            # Target Algorithm Directory (only relevant for SMAC)
            file_format="APT",
            autopytorch=autonet_bundle,
            verbose="DEBUG")

DEBUG:matplotlib.pyplot:Loaded backend agg version unknown.


Q: should CAVE even get an autonet-instance? is all relevant information saved with info about the autonet-instance? would be nicer if there simply was some sort of scenario-file (which is partly/mostly covered by the results-dump)

In [7]:
#cave.apt_overview()
cave.compare_default_incumbent()

ERROR:cave.cavefacade.CAVE:max() arg is an empty sequence
Traceback (most recent call last):
  File "/home/shuki/Repos/CAVE/cave/cavefacade.py", line 55, in wrap
    analyzer = f(self, *args, **kw)
  File "/home/shuki/Repos/CAVE/cave/cavefacade.py", line 268, in compare_default_incumbent
    return CompareDefaultIncumbent(self.runscontainer)
  File "/home/shuki/Repos/CAVE/cave/analyzer/compare_default_incumbent.py", line 21, in __init__
    incumbent = max(incumbents, key=lambda key: incumbents[key])
ValueError: max() arg is an empty sequence
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of emcee
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of skopt
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of pyDOE
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of emcee
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of skopt
DEBUG:lazy_import:Getting attr __file__ of LazyModule instance of pyDOE


ValueError: max() arg is an empty sequence

Other analyzers also run on the APT-data:

In [ ]:
cave.apt_tensorboard()